In [1]:
import scanpy as sc

counts = sc.datasets.pbmc3k().X.todense().T
%load_ext rpy2.ipython

In [2]:
%%R -i counts

library(tidyverse)
library(Seurat)
library(scDEED)

perplexity <- 40
data <- CreateSeuratObject(counts) |>
    FindVariableFeatures() |>
    NormalizeData() |>
    ScaleData() |>
    RunPCA() |>
    RunTSNE(perplexity=perplexity)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp
‘SeuratObject’ was built under R 4.5.0 but the current version is
4.5.1; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

Attaching package: ‘SeuratObject’

The following objects are masked from ‘package:base’:

    intersect, t


Attaching package: ‘scDEED’

The following object is masked from ‘package:stats’:

    optimize

Finding variable features for layer counts
Calculating gene variances
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Calculating feature variances of standardized and clipped values
0%   10   20   30   40   50   60   70   80   90   100%
[----|----|----|----|----|----|----|----|----|----|
**************************************************|
Normalizing layer: counts
Performing log-normalization
0%   10   20   30   40   50   60   70   80   90   100%
[---

In [3]:
%%R

embeddings <- Embeddings(data, "tsne")
normalized_counts <- GetAssayData(data, layer = "scale.data") |>
    as.matrix() |>
    t()

In [4]:
import numpy as np
import rpy2.robjects

embeddings = np.array(rpy2.robjects.globalenv['embeddings'])
normalized_counts = np.array(rpy2.robjects.globalenv["normalized_counts"])

In [5]:
%%R

K <- 8
result <- scDEED(data, K = K, reduction.method = 'tsne', rerun = F, perplexity = perplexity)
dubious <- result$full_results |>
    filter(perplexity == perplexity) |>
    pull(dubious_cells) |>
    str_split(",")

dubious <- as.integer(dubious[[1]])

data <- FindNeighbors(data, features = VariableFeatures(data), k.param = 50)
G <- data@graphs$RNA_nn
N <- map(dubious, ~ which(G[.x, ] > 0)) |>
    set_names(dubious)

[1] "Permuting data"
  |======================================================================| 100%[1] "Permutation finished"


Computing nearest neighbor graph
Computing SNN
In addition: Warning messages:
1: The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the scDEED package.
  Please report the issue to the authors.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
2: The `slot` argument of `SetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.
ℹ The deprecated feature was likely used in the scDEED package.
  Please report the issue to the authors.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 


In [6]:
from distortions.geometry import Geometry, bind_metric, local_distortions

geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": 20}, adjacency_kwds={"radius": 50}, laplacian_kwds={"scaling_epps": 5})
H, Hvv, Hs = local_distortions(embeddings, normalized_counts, geom)
embeddings = bind_metric(embeddings, Hvv, Hs)
N = rpy2.robjects.globalenv["N"]
N_dict = {int(key): list(val) for key, val in N.items()}

In [7]:
from distortions.visualization import dplot

plots = {}
plots["scdeed_distort"] = dplot(embeddings, width=440, height=440)\
    .mapping(x="embedding_0", y="embedding_1")\
    .geom_ellipse(radiusMax=10, radiusMin=1)\
    .inter_edge_link(N=N_dict, stroke="#F25E7A", highlightColor="#C83F58", strokeWidth=.4, highlightStrokeWidth=5, threshold=10, backgroundOpacity=0.5)

In [8]:
#[p.save(f"../paper/figures/{k}.svg") for k, p in plots.items()]

In [9]:
[display(p) for p in plots.values()]

dplot(dataset=[{'embedding_0': 5.783105882455072, 'embedding_1': -12.634558999702302, 'x0': -0.303412665270283…

[None]